In [1]:
# imports
from tkinter import ttk
import tkinter as tk
from PIL import ImageTk, Image
from io import BytesIO
import requests
import json
import csv
import pandas as pd

In [ ]:
# functions to call in the GUI (add these into the gui)

# search button command (might use ingredient input function instead)
def ingredient():


# add button command (we might have to get rid of this part)
def add_ingredient():
    ingredient_list = []



def user_input():
    ingredient_list = []
    while True:
        users_input = input("Do you want to add an ingredient to your list? y/n: ").lower()
        if users_input == "y":
            ingredient = input("Add an ingredient: ").lower()
            ingredient_list.append(ingredient)
        elif users_input == "n":
            break
        else:
            print("Please enter 'y' or 'n'!")

# backend function to add csvs
def search_recipes(ingredient):
    csv_file = open("recipes.csv", "w", newline="", encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Recipe Name", "Thumbnail Link", "Ingredients", "Cook Time (in minutes)", "Diet", "Cuisine", "Recipe Source"])
    
    key = "5cb09004fe27449689868405b5a33789"
    find_ingredients = requests.get(f"https://api.spoonacular.com/recipes/findByIngredients?ingredients={ingredient}&apiKey={key}")
    ingredients_json = find_ingredients.json()
    
    for recipe in ingredients_json:
        id_num = recipe["id"]
        recipe_info = requests.get(f"https://api.spoonacular.com/recipes/{id_num}/information?apiKey={key}").json()
        recipe_name = recipe_info["title"]
        recipe_pic = recipe_info["image"]
        cook_time = recipe_info["readyInMinutes"]
        recipe_diets = recipe_info["diets"]
        if recipe_info["cuisines"]:
            recipe_cuisine = recipe_info["cuisines"]
        else:
            recipe_cuisine = "NaN"
        ingredients_list = []
        for i in recipe_info["extendedIngredients"]:
            ingredient_name = i["nameClean"]
            ingredients_list.append(ingredient_name)
        recipe_source = recipe_info["sourceUrl"]
        csv_writer.writerow([recipe_name, recipe_pic, ingredients_list, cook_time, recipe_diets, recipe_cuisine, recipe_source])
    csv_file.close()

    data = pd.read_csv("recipes.csv")
    recipe_name_list = data["Recipe Name"].tolist()
    img_list = data["Thumbnail Link"].tolist()
    for i in range(len(recipe_name_list)): # add this into the gui
        recipe_box = tk.Frame(recipes, bg="gray", bd=2, padx=10, pady=10)
        recipe_box.pack(side="left", padx=10, pady=10)
        recipe_title = tk.Label(recipe_box, text=recipe_name_list[i], bg="gray")
        recipe_title.pack(anchor="w")
        recipe_response = requests.get(img_list[i])
        info = BytesIO(recipe_response)
        recipe_img = Image.open(info)
        recipe_img.thumbnail((200, 200))
        recipe_pic = ImageTk.PhotoImage(recipe_img)
        recipe_panel = tk.Label(recipe_box, image=recipe_pic, bg="light gray")
        recipe_panel.image = recipe_pic
        recipe_panel.pack(side="center", padx=10)
    

#search_recipes(we would add the .get from the search bar here)

In [3]:
root = tk.Tk()
root.title("Fridge to Food")

mainframe = tk.Frame(root)
mainframe.pack(fill="both", expand=True)

heading = tk.Frame(mainframe, bg="mediumorchid3")
heading.pack(side="top", fill="x")   

response = requests.get("https://i.postimg.cc/tCLRtKB7/logo-1.png")
image_info = BytesIO(response.content)
logo_img = Image.open(image_info)
logo_img.thumbnail((200, 200))
logo = ImageTk.PhotoImage(logo_img)
panel = tk.Label(heading, image=logo, bg="mediumorchid3")
panel.image = logo 
panel.pack(side="left", padx=20)

ingredient_list = []
def add_ingredient(ingredient):
    ingredient_list.append(ingredient)
    ingredients_input.delete("1.0", "end")

def search_recipes(list):
    csv_file = open("recipes.csv", "w", newline="", encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Recipe Name", "Thumbnail Link", "Ingredients", "Cook Time (in minutes)", "Diet", "Cuisine", "Recipe Source"])
    
    key = "5cb09004fe27449689868405b5a33789"
    find_ingredients = requests.get(f"https://api.spoonacular.com/recipes/findByIngredients?ingredients={list}&apiKey={key}")
    ingredients_json = find_ingredients.json()
    
    for recipe in ingredients_json:
        id_num = recipe["id"]
        recipe_info = requests.get(f"https://api.spoonacular.com/recipes/{id_num}/information?apiKey={key}").json()
        recipe_name = recipe_info["title"]
        recipe_pic = recipe_info["image"]
        cook_time = recipe_info["readyInMinutes"]
        recipe_diets = recipe_info["diets"]
        if recipe_info["cuisines"]:
            recipe_cuisine = recipe_info["cuisines"]
        else:
            recipe_cuisine = "NaN"
        ingredients_list = []
        for i in recipe_info["extendedIngredients"]:
            ingredient_name = i["nameClean"]
            ingredients_list.append(ingredient_name)
        recipe_source = recipe_info["sourceUrl"]
        csv_writer.writerow([recipe_name, recipe_pic, ingredients_list, cook_time, recipe_diets, recipe_cuisine, recipe_source])
    csv_file.close()

search = tk.Button(heading, text="🔎", highlightbackground='mediumorchid3', command=lambda:search_recipes(ingredient_list))
search.pack(side="right", padx=20)

add_button = tk.Button(heading, text="✚",highlightbackground='mediumorchid3', command=lambda:add_ingredient(ingredient.get()))
add_button.pack(side="right", padx=10)

ingredient = tk.StringVar()
ingredients_input = tk.Entry(heading, textvariable=ingredient, width=50)
ingredients_input.pack(side="right")

subheading = tk.Frame(mainframe, bg="light gray")
subheading.pack(side="top", fill="x")

instructions = tk.Label(subheading, text="Welcome to Fridge to Food!  Search for ingredients, add the ingredients in your fridge by using ✚, and search for recipes by pressing 🔎!", bg="light gray")
instructions.pack(pady=5)

other_stuff = tk.Label(subheading, text="Each recipe will contain a picture as well as information about its cooktime, ingredients, cuisine, diet, and allergens!", bg="light gray")
other_stuff.pack(pady=5)

more_instructions = tk.Label(subheading, text="↙ You can also filter for specific cook times and cuisines!", bg="light gray")
more_instructions.pack(pady=5)

filters = tk.Frame(mainframe, bg="gray", width=200)
filters.pack(side="left", padx=10, pady=10, fill="y")


filters_canvas = tk.Canvas(filters,bg="gray", width=200,highlightbackground='grey')
filters_canvas.pack(side="left", padx=10, pady=10, fill="y")


filter_scroll = ttk.Scrollbar(filters, orient="vertical",takefocus= True, command=filters_canvas.yview)
filter_scroll.pack(side="right", fill="y")

filters_canvas.configure(yscrollcommand=filter_scroll.set)

frame_inside_canvas = tk.Frame(filters_canvas, bg="gray")
filters_canvas.create_window((0, 0), window=frame_inside_canvas, anchor="nw")

cooktime = tk.Frame(frame_inside_canvas, bg="gray")
cooktime.pack()


cooktime_title = tk.Label(frame_inside_canvas, text="Cook Time (in minutes)", bg="gray", fg="white",font=('Helvetica',15,'underline'))
cooktime_title.pack()

cooktimes = ["<60", ">=60 & <=120", ">120"]
for time in cooktimes:
    cooktime_filter = tk.Checkbutton(frame_inside_canvas, text=time, bg="gray")
    cooktime_filter.pack(anchor="w")

cuisines = tk.Frame(frame_inside_canvas, bg="gray")
cuisines.pack(padx=10, pady=10)

cuisines_label = tk.Label(frame_inside_canvas, text="Cuisines", bg="gray", fg="white",font=('Helvetica',15,'underline'))
cuisines_label.pack(anchor='w')

cuisine_list = ["African", "Asian", "American", "British", "Cajun", "Caribbean", "Chinese",
                "Eastern European", "European", "French", "German", "Greek", "Indian", "Irish", "Italian",
                "Japanese", "Jewish", "Korean", "Latin American", "Mediterranean", "Mexican",
                "Middle Eastern", "Nordic", "Southern", "Spanish", "Thai", "Vietnamese"]

for i in cuisine_list:
    cuisine = tk.Checkbutton(frame_inside_canvas, text=i, bg="gray")
    cuisine.pack(anchor="w")

recipes = tk.Frame(mainframe, bg="white")
recipes.pack(fill="both", padx=10, pady=10)

recipe_heading = tk.Label(recipes, text="Recipes", font=(20), bg="white")
recipe_heading.pack()

data = pd.read_csv("recipes.csv")

recipe_list = data["Recipe Name"].tolist()
img_list = data["Thumbnail Link"].tolist()

def load_img(url):
    response = requests.get(url)
    img_info = BytesIO(response.content)
    pil_img = Image.open(image_info)
    return ImageTk.PhotoImage(pil_img)

def show_img(url):
    global img_refrence
    pil_img = load_img(url)
    if pil_img is not None:
        recipe_img.delete("all")
        recipe_img.create_image(300, 300, anchor="center", image=pil_img)
        img_refrence = pil_img

for i in recipe_list:
    img = 0

    recipe_box = tk.Frame(recipes, bg="gray", bd=2, padx=10, pady=10)
    recipe_box.pack(side="left", padx=10, pady=10)
    recipe_title = tk.Label(recipe_box, text=i, bg="gray")
    recipe_img = tk.Canvas(recipe_box, width=300, height=300, bg="white")
    show_img(img_list[img])

    recipe_img.pack()
    recipe_title.pack(anchor="w")
    print(img_list[img])
    img+=1

def on_frame_configure(event):
    filters_canvas.configure(scrollregion=filters_canvas.bbox("all"))

filters_canvas.bind("<Configure>", on_frame_configure)


root.mainloop()

https://img.spoonacular.com/recipes/607952-556x370.jpg
https://img.spoonacular.com/recipes/607952-556x370.jpg
https://img.spoonacular.com/recipes/607952-556x370.jpg
https://img.spoonacular.com/recipes/607952-556x370.jpg
https://img.spoonacular.com/recipes/607952-556x370.jpg


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\catel\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\catel\AppData\Local\Temp\ipykernel_348\1511790478.py", line 55, in <lambda>
    add_button = tk.Button(heading, text="✚",highlightbackground='mediumorchid3', command=lambda:add_ingredient(ingredient.get()))
                                                                                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\catel\AppData\Local\Temp\ipykernel_348\1511790478.py", line 22, in add_ingredient
    ingredients_input.delete("1.0", "end")
  File "c:\Users\catel\anaconda3\Lib\tkinter\__init__.py", line 3139, in delete
    self.tk.call(self._w, 'delete', first, last)
_tkinter.TclError: bad entry index "1.0"
